# Multitask Discriminator Tests

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import time
from configs.Metric import Metric
from configs.SimulatedData import Proposed

In [ ]:
config= Proposed()

SEQUENCE_LENGTH = config.sequence_lenght_in_sample
GRANUARITY = config.granularity
OVERLAP = config.overlap
BS = config.batch_size
EPOCHS = 50 # config.epochs
NUM_SEQUENCE_TO_GENERATE = config.met_params.sequence_to_generate

SIMULATED_DATA_PATH = "data/simulated_dataset/01 - Source Domain.h5"
SIMULATED_TARGET_DOMAIN = "data/simulated_dataset/output_noise/0.75.h5"

N_SAMPLE_WIENER = SEQUENCE_LENGTH//4
FEAT_WIENER = 2
NOISE_DIM= (N_SAMPLE_WIENER, FEAT_WIENER)

In [ ]:
gpus = tf.config.list_physical_devices('GPU')

if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

gpus

In [ ]:
def convert_dataframe_to_tensorflow_sequences(df:pd.DataFrame, sequence_lenght_in_sample, granularity, shift_between_sequences, batch_size, shuffle=True):
    sequence_lenght = int(sequence_lenght_in_sample*granularity)

    dset = tf.data.Dataset.from_tensor_slices(df.values)
    dset = dset.window(sequence_lenght , shift=shift_between_sequences, stride=granularity).flat_map(lambda x: x.batch(sequence_lenght_in_sample, drop_remainder=True))

    if shuffle:
        dset= dset.shuffle(256)

    dset = dset.batch(batch_size, drop_remainder=True)

    # dset = dset.cache().prefetch(10)

    return dset

In [ ]:
df_d1 = pd.read_hdf(SIMULATED_DATA_PATH)
shape = df_d1.shape[0]
df_d1 = df_d1.iloc[:int(shape/2)]

df_d2 = pd.read_hdf(SIMULATED_TARGET_DOMAIN)
shape = df_d2.shape[0]
df_d2 = df_d2.iloc[:int(shape/2)]

dset_style1 = convert_dataframe_to_tensorflow_sequences(
    df_d1, 
    SEQUENCE_LENGTH, 
    GRANUARITY, 
    int(OVERLAP* SEQUENCE_LENGTH),
    BS,
    shuffle=False
)
dset_style1 = dset_style1.map(lambda batch: (batch, tf.zeros(BS, 1)), num_parallel_calls=tf.data.AUTOTUNE).cache()

dset_style2 = convert_dataframe_to_tensorflow_sequences(
    df_d2, 
    SEQUENCE_LENGTH, 
    GRANUARITY, 
    int(OVERLAP* SEQUENCE_LENGTH),
    BS,
    shuffle=False
)
dset_style2 = dset_style2.map(lambda batch: (batch, tf.zeros(BS, 1) +1), num_parallel_calls=tf.data.AUTOTUNE).cache()

dset_style1= dset_style1.unbatch()
dset_style2= dset_style2.unbatch()

dset = tf.data.Dataset.sample_from_datasets([dset_style1, dset_style2])

dset= dset.batch(BS, drop_remainder=True)

In [ ]:
_, labels = next(iter(dset))

In [ ]:
labels

# Get Sequences from dataset.

In [ ]:
seq_s1, _ = next(iter(dset_style1))
seq_s2, _ = next(iter(dset_style2))


plt.figure(figsize=(18, 10))

ax = plt.subplot(211)
ax.set_title('Style 1 Sequence.')
plt.plot(seq_s1)
plt.grid()

ax = plt.subplot(212)
ax.set_title('Style 2 Sequence.')
plt.plot(seq_s2)
plt.grid()

plt.show()

# Implement models

In [ ]:
# Define AdaIN Layers for Time Series
class AdaIN(tf.keras.layers.Layer):
    def __init__(self):
        super(AdaIN, self).__init__()

    def get_mean_std(self, x, eps=1e-5):
        _mean, _variance = tf.nn.moments(x, axes=[1], keepdims=True)
        standard_dev = tf.sqrt(_variance+ eps)
        return _mean, standard_dev

    def call(self, content_input, style_input):
        # print(content_input.shape, style_input.shape)
        content_mean, content_std = self.get_mean_std(content_input)
        style_mean, style_std = self.get_mean_std(style_input)
        adain_res =style_std* (content_input - content_mean) / content_std+ style_mean
        return adain_res

In [ ]:
def make_decoder(n_sample_wiener:int, feat_wiener:int, style_vector_size:int, out_feat:int):
    _content_input = tf.keras.Input((n_sample_wiener, feat_wiener), name="Content Time Series")
    _style_input = tf.keras.Input((style_vector_size, 1), name="Style Input") 
    _style_input = tf.keras.layers.Flatten()(_style_input)

    stage_1_style = tf.keras.layers.Dense(16, name='1')(_style_input)
    stage_1_style = tf.keras.layers.Reshape((16, 1))(stage_1_style)

    # stage_2_style = tf.keras.layers.Dense(32, name='2')(_style_input)
    # stage_2_style = tf.keras.layers.Reshape((32, 1))(stage_2_style)


    # mini content encoding.
    _content_encoded = tf.keras.layers.Conv1D(128, 5, 1, padding='same', activation="relu")(_content_input)
    _content_encoded = tf.keras.layers.Conv1D(128, 5, 1, padding='same', activation="relu")(_content_encoded)



    x = AdaIN()(_content_encoded, stage_1_style)
    x = tf.keras.layers.Conv1D(256, 5, 1, padding='same', activation="relu")(x)
    # x = AdaIN()(x, stage_1_style)
    x = tf.keras.layers.Conv1D(256, 5, 1, padding='same', activation="relu")(x)
    # x = AdaIN()(x, stage_1_style)
    x = tf.keras.layers.Conv1D(256, 5, 1, padding='same', activation="relu")(x)
    x = tf.keras.layers.Conv1DTranspose(128, 5, 2, padding='same')(x)
    x = tf.keras.layers.LeakyReLU()(x)

    # x = AdaIN()(x, stage_2_style)
    x = tf.keras.layers.Conv1D(512, 5, 1, padding='same', activation="relu")(x)
    # x = AdaIN()(x, stage_2_style)    
    x = tf.keras.layers.Conv1D(512, 5, 1, padding='same', activation="relu")(x)
    # x = AdaIN()(x, stage_2_style)
    x = tf.keras.layers.Conv1D(512, 5, 1, padding='same', activation="relu")(x)
    x = tf.keras.layers.Conv1DTranspose(out_feat, 5, 2, padding='same', activation="linear")(x)
    

    model = tf.keras.Model([_content_input, _style_input], x)
    return model


def make_multitask_discriminator(seq_length:int, n_feat:int, n_style_class:int):
    _input = tf.keras.Input((seq_length, n_feat))
    x = tf.keras.layers.Conv1D(64, 5, 2, padding='same')(_input)
    x = tf.keras.layers.LeakyReLU()(x)
    x = tf.keras.layers.Conv1D(64, 3, 1, padding='same')(x)
    x = tf.keras.layers.LeakyReLU()(x)

    x = tf.keras.layers.Conv1D(128, 5, 2, padding='same')(x)
    x = tf.keras.layers.LeakyReLU()(x)
    x = tf.keras.layers.Conv1D(128, 3, 1, padding='same')(x)
    x = tf.keras.layers.LeakyReLU()(x)

    flattened = tf.keras.layers.Flatten()(x)
    _realness_output = tf.keras.layers.Dense(32, activation="relu")(flattened)
    # _realness_output = tf.keras.layers.Dense(32, activation="relu")(_realness_output)
    _realness_output = tf.keras.layers.Dense(1, activation="sigmoid")(_realness_output)

    _style_recognition_output = tf.keras.layers.Dense(n_style_class, activation='softmax')(flattened)

    model = tf.keras.Model(_input, [_realness_output, _style_recognition_output])
    early_discriminator = tf.keras.Model(_input, flattened)

    return model, early_discriminator

In [ ]:
def wiener_process(batch:int, n_sample_wiener:int, n_feat_wiener:int):
    d_noise = tf.random.normal([batch, n_sample_wiener, n_feat_wiener])
    wiener_noise = tf.math.cumsum(d_noise, axis=1)
    return wiener_noise

seed = wiener_process(NUM_SEQUENCE_TO_GENERATE, N_SAMPLE_WIENER, FEAT_WIENER)

# Test if the generator works well

In [ ]:
generator = make_decoder(N_SAMPLE_WIENER, FEAT_WIENER, 1, df_d2.shape[1])
discriminator, feature_projector= make_multitask_discriminator(SEQUENCE_LENGTH, df_d2.shape[1], 2)

In [ ]:
discriminator.summary()

In [ ]:
generator.summary()
tf.keras.utils.plot_model(generator, "model.png", show_shapes=True)

In [ ]:
tf.keras.utils.plot_model(discriminator, show_shapes=True)

In [ ]:
generated_sequences = generator([seed, tf.zeros(NUM_SEQUENCE_TO_GENERATE, 1)+1])

In [ ]:
plt.figure(figsize=(18, 10))
plt.plot(generated_sequences[0])
plt.grid()
plt.show()

# Define losses

In [ ]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=False)
error_classif = tf.keras.losses.SparseCategoricalCrossentropy()

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

def style_classsification_loss(y_pred, y_true):
    return error_classif(y_true, y_pred)

def similarity_loss(extracted_features:np.ndarray):
    anchor = extracted_features[0]
    return tf.exp(-(tf.norm(extracted_features[1:]- anchor)))

# Define Correlation metric

In [ ]:
def optimized_cov(a,v):
    nan_mean_a = np.nanmean(a, axis=1).reshape((-1,1))
    nan_mean_b = np.nanmean(v, axis=1).reshape((-1,1))
    return np.nansum((a- nan_mean_a)* (v- nan_mean_b), axis=1)

def mean_difference(a,v):
    return np.nanmean(a) - np.nanmean(v)

def optimized_windowed_cov(a, v, beta=Metric.mean_senssibility_factor):
    if a.shape[1] > v.shape[1]:
        _a, _v = v, a 
    else: 
        _a, _v = a, v

    n = _a.shape[1]
    corrs = []

    for k in range(_v.shape[1] - _a.shape[1]):
        __v = _v[:, k: n+k]
        # Compute the covariance 
        augmented_cov = optimized_cov(_a,__v)+ beta* mean_difference(_a,__v)

        corrs.append(augmented_cov)
        
    return np.array(corrs)

def signature_on_batch(x:np.ndarray, ins:list, outs:list, sig_seq_len:int):
    """Compute the signature from a given batch of MTS sequences `x`

    Args:
        x (np.ndarray): the batch
        ins (list): input columns
        outs (list): output label solumns
        sig_seq_len (int): the desired signature length

    Returns:
        np.ndarray: the min, max, mean signature.
    """
    sigs = []
    shift = sig_seq_len//2
    childrens = x[:, shift:-shift]

    for _in in ins:
        for _out in outs:
            c1 = x[:, :, _in]
            c2 = childrens[:, :, _out]
            
            sig = optimized_windowed_cov(c1, c2)

            sigs.append(sig)

    mins = np.min(sigs, axis=-1)
    maxs = np.max(sigs, axis=-1)
    means= np.mean(sigs, axis=-1)

    signatures = np.stack([mins, maxs, means], axis=-1)

    return signatures

def signature_metric(source_sig:np.ndarray, target_sig:np.ndarray):
    # Shape: (n_features, sign_seq_lenght, 3)
    min_source = source_sig[0]
    max_source = source_sig[1]
    mean_source = source_sig[2]

    min_target = target_sig[0]
    max_target = target_sig[1]
    mean_target = target_sig[2]

    mean_differences = np.mean(mean_target- mean_source)
    area_source = np.mean(max_source- min_source)
    area_target = np.mean(max_target- min_target)

    met = np.power(mean_differences, 2) + Metric.noise_senssitivity*np.power(area_target- area_source, 2)

    return met

In [ ]:
batched_style1 = dset_style1.batch(BS)
batched_style2 = dset_style2.batch(BS)

style_1_seq, _ = next(iter(batched_style1))
style_1_signature = signature_on_batch(style_1_seq, config.met_params.ins, config.met_params.outs, config.met_params.signature_length)

style_2_seq, _ = next(iter(batched_style2))
style_2_signature = signature_on_batch(style_2_seq, config.met_params.ins, config.met_params.outs, config.met_params.signature_length)

# Custom Training loop

In [ ]:
generator_optimizer = tf.keras.optimizers.RMSprop()
discriminator_optimizer = tf.keras.optimizers.RMSprop(0.0005)  

generator_metric = tf.keras.metrics.Mean() # Total Generator metric.
style_loss_of_generations_metric = tf.keras.metrics.Mean()
discriminator_on_fake_metric = tf.keras.metrics.Mean()

realness_metric = tf.keras.metrics.Mean() # Discriminator loss...
style_classif_metric = tf.keras.metrics.Mean()

style1_corr_metric = tf.keras.metrics.Mean()
style2_corr_metric = tf.keras.metrics.Mean()

sim_loss_metric = tf.keras.metrics.Mean()


In [ ]:
from datetime import datetime
import io

date_str = datetime.now().strftime('%Y-%m-%d_%H_%M_%S')

BASE_DIR = f"logs/{date_str} - Multi task discriminator"
TRAIN_LOGS_DIR_PATH = f"{BASE_DIR}/fit"
GENERATION_LOG = f"{BASE_DIR}/Generations"

TRAIN_SUMMARY_WRITER = tf.summary.create_file_writer(TRAIN_LOGS_DIR_PATH)

def plot_to_buff(generations_style1:np.ndarray, generations_style2:np.ndarray, nvertical:int=4):
    legend = [f"feat {j}" for j in range(generations_style1.shape[-1])]

    fig = plt.figure(figsize=(18, 10))
    plt.suptitle("Generations After GAN Training.")

    for i in range(nvertical):
        ax = plt.subplot(nvertical, 2, 2* i+ 1)
        ax.set_title(f"Sequence {i+ 1} Style 1")

        plt.plot(generations_style1[i])
        ax.grid(True)
        plt.legend(legend)

        ax = plt.subplot(nvertical, 2, 2* i+ 2)
        ax.set_title(f"Sequence {i+ 1} Style 2")

        plt.plot(generations_style2[i])
        ax.grid(True)
        plt.legend(legend)

    plt.tight_layout()

    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    plt.close(fig)
    return buf


def log_losses(epoch, plot_buf):

    image = tf.image.decode_png(plot_buf.getvalue(), channels=4)
    image = tf.expand_dims(image, 0)


    with TRAIN_SUMMARY_WRITER.as_default():
        tf.summary.scalar("0 - Style 1 Correlation Metric", style1_corr_metric.result(), step=epoch)
        tf.summary.scalar("0 - Style 2 Correlation Metric", style2_corr_metric.result(), step=epoch)

        tf.summary.scalar("1 - Total Generator Loss", generator_metric.result(), step=epoch)
        tf.summary.scalar("1 - Discriminator on Generations", discriminator_on_fake_metric.result(), step=epoch)
        tf.summary.scalar("1 - Style Loss on Generations", style_loss_of_generations_metric.result(), step=epoch)

        tf.summary.scalar("2 - Discriminator Loss", realness_metric.result(), step=epoch)
        tf.summary.scalar("3 - Style Classification Loss", style_classif_metric.result(), step=epoch)

        tf.summary.scalar("4 - Is Colapsing", sim_loss_metric.result(), step=epoch)
        
        tf.summary.image("Training Generations", image, step=epoch)

In [ ]:
def style_is_ok(labels):
    return labels[labels == 1].shape[0] > 0 and labels[labels == 0].shape[0] > 0

In [ ]:
def train_step(real_sequences, style_labels):
    noise= wiener_process(BS, N_SAMPLE_WIENER, FEAT_WIENER)


    with tf.GradientTape() as gen_tape:
        gen_sequences = generator([noise, style_labels])
        pred_on_fake, classif_on_fake= discriminator(gen_sequences)
        extracted_feat = feature_projector(gen_sequences)

        gen_loss= generator_loss(pred_on_fake)
        style_on_fake= style_classsification_loss(classif_on_fake, style_labels)

        feature_style1 = extracted_feat[style_labels == 0.]
        feature_style2 = extracted_feat[style_labels == 1.]

        sim_loss_style_1 = similarity_loss(feature_style1)
        sim_loss_style_2 = similarity_loss(feature_style2)

        sim_loss= (sim_loss_style_1+ sim_loss_style_2)/ 2

        final_gen_loss = gen_loss+ style_on_fake + sim_loss

    with  tf.GradientTape() as disc_tape:
        pred_on_real, classif_on_real= discriminator(real_sequences)
        pred_on_fake, classif_on_fake= discriminator(gen_sequences)

        dis_loss = discriminator_loss(pred_on_real, pred_on_fake)
        style_on_real= style_classsification_loss(classif_on_real, style_labels)

    discr_grads= disc_tape.gradient([dis_loss, style_on_real], discriminator.trainable_variables)
    gen_grads= gen_tape.gradient(final_gen_loss, generator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gen_grads, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(discr_grads, discriminator.trainable_variables))

    generator_metric(final_gen_loss)
    style_loss_of_generations_metric(style_on_fake)
    discriminator_on_fake_metric(gen_loss)
    

    realness_metric(dis_loss)
    style_classif_metric(style_on_real)
    sim_loss_metric(sim_loss)



def train():
    n_steps = "?"
    for e in range(EPOCHS):
        generator_metric.reset_states()
        realness_metric.reset_states()
        style_classif_metric.reset_states()
        style1_corr_metric.reset_states()
        style2_corr_metric.reset_states()
        sim_loss_metric.reset_states()

        for b, (sequence, style_label) in enumerate(dset):
            if style_is_ok(style_label):
                train_step(sequence, style_label)
                print(f"\r EPOCH {e} [{b}/{n_steps}]\
 Total Gen Loss: {generator_metric.result():0.4f};\
 Style classification on Gen: {style_loss_of_generations_metric.result():0.4f};\
 D on Generations: {discriminator_on_fake_metric.result():0.4f};\
 Similarity loss: {sim_loss_metric.result():0.4f};\
 D Loss: {realness_metric.result():0.4f};\
 Style classif loss: {style_classif_metric.result():0.4f}", end="")
        print()

        if e == 0:
            n_steps = b

        # Compute our metric.
        gen_style1 = generator([seed, tf.zeros(NUM_SEQUENCE_TO_GENERATE, 1)], training=False)
        gen_style2 = generator([seed, tf.zeros(NUM_SEQUENCE_TO_GENERATE, 1)+ 1], training=False)

        gen_style1_signature= signature_on_batch(gen_style1, config.met_params.ins, config.met_params.outs, config.met_params.signature_length)
        gen_style2_signature= signature_on_batch(gen_style2, config.met_params.ins, config.met_params.outs, config.met_params.signature_length)

        style1_sig_diff= signature_metric(style_1_signature, gen_style1_signature)
        style2_sig_diff= signature_metric(style_2_signature, gen_style2_signature)

        style1_corr_metric(style1_sig_diff)
        style2_corr_metric(style2_sig_diff)

        print(f" [SIGNATURE DIFFERENCE]: Style 1: {style1_sig_diff:0.4f}; Style 2: {style2_sig_diff:0.4f}")

        buff = plot_to_buff(gen_style1, gen_style2)


        log_losses(e, buff)

In [ ]:
train()

In [ ]:
generated_sequences = generator([seed, tf.zeros(BS, 1)])

plt.figure(figsize=(18, 10))
plt.plot(generated_sequences[0])
plt.grid()
plt.show()

In [ ]:
generated_sequences = generator([seed, tf.zeros(BS, 1)+ 1])

plt.figure(figsize=(18, 10))
plt.plot(generated_sequences[0])
plt.grid()
plt.show()